In [432]:
import numpy as np
import pandas as pd
import time
import random
import matplotlib.pyplot as plt
import math

In [3]:
df_distancias = pd.read_csv("Caso II - Matriz de Distancias.csv", delimiter= ",", index_col= 0)
dist_matrix = df_distancias.values
ubicacion_clientes = pd.read_csv("Caso II - Ubicacion Clientes.csv", delimiter= ",", index_col= 0)
datos_prueba = pd.read_csv("Caso II - Set de Datos.csv", delimiter= ",", index_col= 0)
test_data = datos_prueba.values

In [21]:
column_mapping = {column_name: int(column_name) for column_name in datos_prueba.columns}

# Reassign columns with new integer names
datos_prueba.rename(columns=column_mapping, inplace=True)

In [24]:
datos_prueba.loc[['A', 'B'],1]

Cliente
A    4
B    6
Name: 1, dtype: int64

In [4]:
#DEFINO LA MATRIZ LLEGA
def llega(x):
   return True if x <= 2 else False

df_llega = df_distancias.applymap(llega)

In [124]:
#DEFINIMOS CONSTANTES
#PROBLEMA
TARIFA_PROMEDIO = 1200
AHORRO = 0.05
TARIFA_OBJETIVO = TARIFA_PROMEDIO * (1-AHORRO)
CANTIDAD_DE_CLIENTES_POR_CAMION = 3
CANTIDAD_DE_CAMIONES = 6
TONS_MAX_POR_CAMION = 12
#METAHEURISTICA
CANTIDAD_DE_PEDIDOS_CAMBIAR = 3


In [6]:
class Camion:
    def __init__(self, pedidos):
        self.pedidos = pedidos #El camion contiene los pedidos que son un series
   
    def tons(self): #Calcula las toneladas que tiene el camión
        tons = 0
        for ton in self.pedidos.values:
            tons += ton
        return tons
    def llega_clientes(self):
        clientes = self.pedidos.index.to_list()
        

    def costo(self): #Computa el costo de forma que en caso de tener >12 tons se cobra 3mil por tonelada extra
        tons = self.tons()
        costo = float(0)
        if (tons == 0):
            costo = 5000
        elif (tons < 4):
            costo = 5600
        elif (tons < 6.5):
            costo = tons * 1400
        elif (tons < 9.5):
            costo = tons * 1200
        elif (tons < 12):
            costo = tons * 1000
        else:
            costo = 12000 + (tons - 12)  * 3000 
            #self.tons = self.tons - tons#ADEMÁS RESTAR A LAS TONELADAS TOTALES LAS NO RUTEADAS
        return costo

In [40]:
#ASIGNA LOS CLIENTES ALEATORIAMENTE A LOS CAMIONES SIN REVISAR SI PUEDEN REALIZAR EL PEDIDO O NO
#AGREGAR PARA QUE CHEQUEE SI LOS PUEDE REPARTIR O NO!!!
#clientes = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L'] #Lista con los clientes para ese día
dia = 1
pedidos_por_asignar = datos_prueba[datos_prueba[dia] > 0].index
pedidos_por_asignar = pedidos_por_asignar.tolist()
pedidos_por_asignar.pop()
print(pedidos_por_asignar)
#Ver dependiendo de la cantida de camiones
#Reemplazar por un for con la cantidad 
 #inicializo los clientes para cada camión
clientes_camion = [[] for _ in range(CANTIDAD_DE_CAMIONES)] #Armo la lista con los pedidos de los camiones por cliente
pedidos_no_asignados = []
#q = len(clientes) #Guardo la cantidad de clientes en una variable q para usar en el range del for
#-------------------------------------------------------------------------
#For para asignar los pedidos aleatoriamente
for i in range(len(pedidos_por_asignar)): #Recorro toda la lista de clientes para ese dia
    cliente_a_asignar = pedidos_por_asignar.pop() #Elimino el ultimo cliente de la lista de clientes a la vez que le asigno el cliente eliminado a la variable cliente
    rand = random.randint(0, CANTIDAD_DE_CAMIONES-1) #genero un numero aleatorio entero del 0 al 5
    camion_cliente = clientes_camion[rand]
    cantidad_clientes = len(camion_cliente)
    if cantidad_clientes < 3: #Verifico que el camion no tenga 3 clientes
        llega_list = []
        for j in range(cantidad_clientes):
            cliente_en_camion = camion_cliente[j]
            llega = df_llega.loc[cliente_en_camion,cliente_a_asignar]
            llega_list.append(llega)
        llega_array = np.array(llega_list)
        llega_todos = np.prod(llega_array)
        #print(llega_list)
        #print(prod)
        if(llega_todos):
            clientes_camion[rand].append(cliente_a_asignar)
        else:
            pedidos_no_asignados.append(cliente_a_asignar)
print("Asignacion de cliente por camión:", clientes_camion)
#-----------------------------------
camiones = [] #Inicializo las lista de los camiones

for camion in clientes_camion: #Recorro los clientes por camion
    pedidos = datos_prueba.loc[camion,dia] #Selecciono los pedidos de ese cliente(las tons)
    camiones.append(Camion(pedidos)) #Creo un camion nuevo con ese pedido y lo agrego a la lista de clientes

tons_camiones = np.array([])
for camion in camiones:
    tons_camiones = np.append(tons_camiones, camion.tons())



tons_repartidas = np.minimum(tons_camiones, np.full((6,), 12))
tot_tons_repartidas = float(np.sum(tons_repartidas))

print("Toneladas asignadas : ", tons_camiones )
print("Toneladas repartidas: ", tons_repartidas)
costo_camiones = np.array([]) #Creo un array vacio para llenar con el costo de los camiones

for camion in camiones: #Calculo el costo por camión
    costo_camiones = np.append(costo_camiones,camion.costo())

print(costo_camiones) #imprimo el array del costo por camion
costo_total = np.array(costo_camiones).sum() #Asigno Costo total diario
print(costo_total) #Imprimo el costo total diario

tons_totales = datos_prueba.loc['Total', [dia]] #Selecciono la toneladas totales para ese dia

costo_por_ton = costo_total/tot_tons_repartidas #Calculo el costo por tonelada
    
print("El costo por tonelada es: ", int(costo_por_ton)) #Imprimo le costo por tonelada

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L']
Asignacion de cliente por camión: [['L', 'F'], ['J', 'B'], ['H', 'G'], ['I', 'E'], ['C'], ['K']]
Toneladas asignadas :  [9. 7. 8. 9. 1. 1.]
Toneladas repartidas:  [9. 7. 8. 9. 1. 1.]
[10800.  8400.  9600. 10800.  5600.  5600.]
50800.0
El costo por tonelada es:  1451


In [504]:
#FUNCIONES
dia = 1
#generarSolucionInicial
def generarSolucionInicial(pedidos): #PEDIDOS ES UNA PANDAS SERIES CON LOS PEDIDOS
    #-----------------------CAMBIAR DATOS_PRUEBA 2 ----------------------------------------------------------------------
    pedidos = pedidos[pedidos > 0 ].index
    pedidos = pedidos.to_list()
    pedidos.pop()
    pedidos_por_asignar = pedidos
    clientes_por_camion = [[] for _ in range(CANTIDAD_DE_CAMIONES)] #Armo la lista con los pedidos de los camiones por cliente
    pedidos_no_asignados = []
    cantidad_clientes = len(pedidos_por_asignar)
    for i in range(cantidad_clientes): #Recorro toda la lista de clientes para ese dia
        cliente = pedidos_por_asignar.pop() #Elimino el ultimo cliente de la lista de clientes a la vez que le asigno el cliente eliminado a la variable cliente
        rand = random.randint(0, CANTIDAD_DE_CAMIONES-1) #genero un numero aleatorio entero del 0 al 5
        camion_cliente = clientes_por_camion[rand]
        cantidad_clientes = len(camion_cliente)
        if cantidad_clientes < CANTIDAD_DE_CLIENTES_POR_CAMION: #Verifico que el camion no tenga 3 clientes
            llega_list = []
            for j in range(cantidad_clientes):
                cliente_en_camion = camion_cliente[j]
                llega = df_llega.loc[cliente_en_camion,cliente]
                llega_list.append(llega)
            llega_array = np.array(llega_list)
            llega_todos = np.prod(llega_array)
            if(llega_todos):
                clientes_por_camion[rand].append(cliente)
            else:
                pedidos_no_asignados.append(cliente)
        else:
            pedidos_no_asignados.append(cliente)
    return clientes_por_camion, pedidos_no_asignados

def pedidosPorCamion(clientes_por_camion, pedidos): #RECIBE UNA LISTA Y DEVUELVE UNA LIST DE SERIES
    pedidos_por_camion = []
    for camion in clientes_por_camion: #Recorro los clientes por camion
            pedido = pedidos[camion] #Selecciono los pedidos de ese cliente(las tons)
            pedidos_por_camion.append(pedido)
    return pedidos_por_camion

#Calcular costo de la solucion
def tonsAsignadasPorCamion(pedidos_por_camion): #RECIBE UNA LIST DE SERIES Y DEVULEVE UNA LISTA DE NUMEROS
    tons = 0
    tons_repartidas_por_camion = []
    toneladas_no_repartidas = 0
    for pedido in pedidos_por_camion:
        tons = pedido.values.sum()
        if(tons <= TONS_MAX_POR_CAMION):
            tons_repartidas_por_camion.append(tons)
        else:
            tons_repartidas_por_camion.append(TONS_MAX_POR_CAMION)
            toneladas_no_repartidas += tons-TONS_MAX_POR_CAMION
    return tons_repartidas_por_camion, toneladas_no_repartidas


def costoPorCamion(tons_repartidas_por_camion):
    costo_por_camion = []
    for tons in tons_repartidas_por_camion:
        costo = float(0)
        if (tons == 0):
            costo = 5000
        elif (tons < 4):
            costo = 5600
        elif (tons < 6.5):
            costo = tons * 1400
        elif (tons < 9.5):
            costo = tons * 1200
        else:
            costo = tons * 1000
        costo_por_camion.append(costo)
    return costo_por_camion

def tonsNoAsignadas(pedidos_no_asignados, pedidos):
    return pedidos[pedidos_no_asignados].values.sum();

def costoTotal(costo_por_camion, tons_no_asignadas_o_no_repartidas):
    costo_total = np.array(costo_por_camion).sum() + tons_no_asignadas_o_no_repartidas * 3000
    return costo_total

def tonsRepartidas(tons_repartidas_por_camion):
    tons_repartidas = np.array(tons_repartidas_por_camion).sum()
    return tons_repartidas


In [419]:
def generarVecino1(clientes_por_camion, pedidos_no_asignados): #Recibe una list de lists y una list
    pedidos_por_asignar = pedidos_no_asignados
    pedidos_no_asignados = []
    #---------SACA UN PEDIDO POR CAMION----------
    for i in range(CANTIDAD_DE_CAMIONES):
        clientes_por_camion_i = clientes_por_camion[i]#Seleccionamos los pedidos del camion i
        len_i = len(clientes_por_camion_i) #Veo cuantos pedidos tiene el camion
        if(len_i > 0):
            rand = random.randint(0, len_i-1) #Selecciono el pedido a cambiar
            cliente_eliminado = clientes_por_camion_i.pop(rand)
            #print("Cliente eliminado: ", cliente_eliminado)
            pedidos_por_asignar.append(cliente_eliminado)
    #--------------CHEQUADO OK-------------------
    #------AGREGA LOS PEDIDOS ALEATORIAMENTE A LOS CAMIONES------
    cantidad_pedidos_por_asignar = len(pedidos_por_asignar)
    for i in range(cantidad_pedidos_por_asignar):
        cliente_a_asignar = pedidos_por_asignar.pop()
        rand = random.randint(0, CANTIDAD_DE_CAMIONES-1)
        camion = clientes_por_camion[rand]
        cantidad_clientes = len(camion)
        if  cantidad_clientes < CANTIDAD_DE_CLIENTES_POR_CAMION:
            llega_list = []
            for j in range(cantidad_clientes):
                cliente_en_camion = camion[j]
                llega = df_llega.loc[cliente_en_camion,cliente_a_asignar]
                llega_list.append(llega)
            llega_array = np.array(llega_list)
            prod = np.prod(llega_array)

            if(prod):
                clientes_por_camion[rand].append(cliente_a_asignar)
            else:
                pedidos_no_asignados.append(cliente_a_asignar)
        else:
            pedidos_no_asignados.append(cliente_a_asignar)
    return  clientes_por_camion, pedidos_no_asignados #Devuelve una list de lists y una list


In [86]:
clientes_camion, pedidos_no_asignados = buscarVecino1(clientes_camion, pedidos_no_asignados)

print("clientes_camion:", clientes_camion)
print("pedidos_no_asignados:", pedidos_no_asignados)

clientes_camion: [[], [], ['B'], ['E', 'D', 'H'], [], ['J', 'K', 'L']]
pedidos_no_asignados: []


In [369]:
pedidos_no_asignados = None
clientes_camion = None

# clientes_camion,pedidos_no_asignados = generarSolucionInicial(1)
# print(pedidos_no_asignados)
# print(clientes_camion)

pedidos = datos_prueba[1]

clientes_por_camion, pedidos_no_asignados = generarSolucionInicial(pedidos)
print("clientes_por_camion:", clientes_por_camion)
print("pedidos_no_asignados:", pedidos_no_asignados)
pedidos_por_camion = pedidosPorCamion(clientes_por_camion, pedidos)
print(pedidos_por_camion)
tons_repartidas_por_camion, tons_no_repartidas = tonsAsignadasPorCamion(pedidos_por_camion = pedidos_por_camion)
print("tons_repartidas_por_camion: ", tons_repartidas_por_camion)
print("toneladas_no_repartidas:", tons_no_repartidas)
tons_no_asignadas = tonsNoAsignadas(pedidos_no_asignados, pedidos)
costo_por_camion = costoPorCamion(tons_repartidas_por_camion)
print("tons_no_asignadas:", tons_no_asignadas)

tons_totales = np.array(tons_repartidas_por_camion).sum() + tons_no_repartidas + tons_no_asignadas
print(tons_totales)
tons_repartidas = np.array(tons_repartidas_por_camion)
tons_no_asignadas_o_no_repartidas = tons_no_repartidas + tons_no_asignadas
costo_por_camion = costoPorCamion(tons_repartidas_por_camion)

print("costo_por_camion:", costo_por_camion)
print("tons_no_asignadas:",tons_no_asignadas_o_no_repartidas)

costo_total = costoTotal(costo_por_camion, tons_no_asignadas_o_no_repartidas)
print('costo_total:', costo_total)

tons_repartidas = tonsRepartidas(tons_repartidas_por_camion)
print(tons_repartidas)

costo_por_ton_repartida = costo_total/tons_repartidas

print(costo_por_ton_repartida)


clientes_por_camion: [[], [], ['J', 'I'], ['L', 'F'], ['K', 'H', 'G'], ['D', 'A']]
pedidos_no_asignados: ['E', 'C', 'B']
[Series([], Name: 1, dtype: int64), Series([], Name: 1, dtype: int64), Cliente
J    1
I    5
Name: 1, dtype: int64, Cliente
L    6
F    3
Name: 1, dtype: int64, Cliente
K    1
H    5
G    3
Name: 1, dtype: int64, Cliente
D    8
A    4
Name: 1, dtype: int64]
tons_repartidas_por_camion:  [0, 0, 6, 9, 9, 12]
toneladas_no_repartidas: 0
tons_no_asignadas: 11
47
costo_por_camion: [5000, 5000, 8400, 10800, 10800, 12000]
tons_no_asignadas: 11
costo_total: 52011
36
1444.75


In [368]:
clientes_por_camion, pedidos_no_asignados = generarVecino1(clientes_por_camion= clientes_por_camion, pedidos_no_asignados= pedidos_no_asignados )

print("clientes_por_camion>", clientes_por_camion)
print("pedidos_no_asignados>", pedidos_no_asignados)

clientes_por_camion> [['G'], ['I'], [], [], ['K'], ['J']]
pedidos_no_asignados> ['C', 'D', 'A']


In [505]:
#HIPERPARÁMETROS DE RECOCIDO SIMULADO
T_INICIAL = 100
T_FINAL = 10
PASO = -10
ITERACIONES_POR_TEMPERATURA = 10
# clientes_camion,pedidos_no_asignados = generarSolucionInicial(1)
# print(pedidos_no_asignados)
# print(clientes_camion)
pedidos = datos_prueba[1]

def recocidoSimulado(pedidos):
    clientes_por_camion_min = []
    pedidos_no_asignados_min = []
    costo_por_ton_repartida_min = float('inf')
    pedidos_no_asignados = None
    #pedidos = datos_prueba[1]

    #GENERO LAS SOLUCION INICIAL
    clientes_por_camion, pedidos_no_asignados = generarSolucionInicial(pedidos)
    pedidos_por_camion = pedidosPorCamion(clientes_por_camion, pedidos)
    tons_repartidas_por_camion, tons_no_repartidas = tonsAsignadasPorCamion(pedidos_por_camion = pedidos_por_camion)
    tons_no_asignadas = tonsNoAsignadas(pedidos_no_asignados, pedidos)
    costo_por_camion = costoPorCamion(tons_repartidas_por_camion)
    tons_no_asignadas_o_no_repartidas = tons_no_repartidas + tons_no_asignadas
    costo_por_camion = costoPorCamion(tons_repartidas_por_camion)
    costo_total = costoTotal(costo_por_camion, tons_no_asignadas_o_no_repartidas)
    tons_repartidas = tonsRepartidas(tons_repartidas_por_camion)
    #CALCULO EL COSTO POR TON REPARTIDA
    costo_por_ton_repartida_1 = costo_total/tons_repartidas
    print('clientes_por_camion:', clientes_por_camion)
    print('pedidos_no_asignados:', pedidos_no_asignados)
    clientes_por_camion_1 = clientes_por_camion
    pedidos_no_asignados_1 = pedidos_no_asignados
    clientes_por_camion_min = clientes_por_camion
    pedidos_no_asignados_min = pedidos_no_asignados
    costo_por_ton_repartida_min = costo_por_ton_repartida_1

    for t in range(T_INICIAL, T_FINAL, PASO):
        print("-----------------Temperatura:", t,"----------------")
        print("costo_por_ton_repartida_min", costo_por_ton_repartida_min)
        print("clientes_por_camion_min:", clientes_por_camion_min)
        print("pedidos_no_asignados_min:", pedidos_no_asignados_min)
        for n in range(ITERACIONES_POR_TEMPERATURA):
            clientes_por_camion, pedidos_no_asignados = generarVecino1(clientes_por_camion_1, pedidos_no_asignados_1)
            print("clientes_por_camion:", clientes_por_camion)
            print("pedidos_no_asignados:", pedidos_no_asignados)
            pedidos_por_camion = pedidosPorCamion(clientes_por_camion, pedidos)
            tons_repartidas_por_camion, tons_no_repartidas = tonsAsignadasPorCamion(pedidos_por_camion = pedidos_por_camion)
            tons_no_asignadas = tonsNoAsignadas(pedidos_no_asignados, pedidos)
            costo_por_camion = costoPorCamion(tons_repartidas_por_camion)
            tons_no_asignadas_o_no_repartidas = tons_no_repartidas + tons_no_asignadas
            costo_por_camion = costoPorCamion(tons_repartidas_por_camion)
            costo_total = costoTotal(costo_por_camion, tons_no_asignadas_o_no_repartidas)
            tons_repartidas = tonsRepartidas(tons_repartidas_por_camion)
            #CALCULO EL COSTO POR TON REPARTIDA
            costo_por_ton_repartida = costo_total/tons_repartidas
            delta = costo_por_ton_repartida_1 - costo_por_ton_repartida #Calculo delta como Sa - Sb
            if delta < 0 : #Si la solucion nueva es peor que la anterio delta < 0
                p = math.e**(delta/t)
                rand_0_1 = random.random()
                print("p:",  f"{p:.2f}") 
                print("rand:", f"{rand_0_1:.2f}")
                if p > rand_0_1 :
                    #ACEPTO UNA PEOR SOLUCION
                    #print("----Tomo solucion peor---")
                    costo_por_ton_repartida_1 = costo_por_ton_repartida
                    pedidos_no_asignados_1 = pedidos_no_asignados
                    clientes_por_camion_1 = clientes_por_camion
                else:
                    #SE QUEDA CON LA SOLUCION ANTERIOR
                    costo_por_ton_repartida = costo_por_ton_repartida_1
                    pedidos_no_asignados = pedidos_no_asignados_1
                    clientes_por_camion = clientes_por_camion_1
            else:
                #ACEPTA LA SOLUCION MEJOR
                costo_por_ton_repartida_1 = costo_por_ton_repartida
                pedidos_no_asignados_1 = pedidos_no_asignados
                clientes_por_camion_1 = clientes_por_camion

            if costo_por_ton_repartida_min > costo_por_ton_repartida:
                #ACTUALIZA A LA MEJOR SOLUCION HASTA AHORA
                costo_por_ton_repartida_min = costo_por_ton_repartida
                clientes_por_camion_min = clientes_por_camion
                pedidos_no_asignados_min = pedidos_no_asignados
            # print("costo_por_ton_repartida:",costo_por_ton_repartida)
            # print("costo_por_ton_repartida_1:", costo_por_ton_repartida_1)
            # print("clientes_por_camion:", clientes_por_camion)
            # print("Pedidos_no_asignados:", pedidos_no_asignados)
            # print("clientes_por_camion_1:", clientes_por_camion_1)
            # print("pedidos_no_asignados_1:", pedidos_no_asignados_1)


    return costo_por_ton_repartida_min, clientes_por_camion_min, pedidos_no_asignados_min

In [507]:
print(costo_por_ton_repartida_min)
print(clientes_por_camion_min)
print(pedidos_no_asignados_min)

1251.4871794871794
[[], ['L', 'B'], [], [], ['F'], ['A']]
[]


In [506]:
costo_por_ton_repartida_min, clientes_por_camion_min, pedidos_no_asignados_min = recocidoSimulado(pedidos)

clientes_por_camion: [['L'], ['K', 'J', 'I'], ['F', 'D'], ['G'], ['C', 'B'], ['H']]
pedidos_no_asignados: ['E', 'A']
-----------------Temperatura: 100 ----------------
costo_por_ton_repartida_min 1251.4871794871794
clientes_por_camion_min: [['L'], ['K', 'J', 'I'], ['F', 'D'], ['G'], ['C', 'B'], ['H']]
pedidos_no_asignados_min: ['E', 'A']
clientes_por_camion: [['I', 'A'], ['K', 'J'], ['F', 'D'], [], ['C', 'L'], ['H', 'B']]
pedidos_no_asignados: ['G', 'E']
p: 0.65
rand: 0.86
clientes_por_camion: [['A', 'I'], ['K'], ['F', 'H', 'L'], [], ['C'], ['B', 'D']]
pedidos_no_asignados: ['J']
p: 0.13
rand: 0.88
clientes_por_camion: [['A'], ['C'], ['F', 'H', 'I'], ['K'], ['L'], ['D', 'B']]
pedidos_no_asignados: []
p: 0.32
rand: 0.15
clientes_por_camion: [[], [], ['H', 'I', 'K'], ['L', 'C'], ['B', 'F', 'A'], ['D']]
pedidos_no_asignados: []
clientes_por_camion: [['L'], [], ['H', 'K'], ['C'], ['B', 'F', 'A'], []]
pedidos_no_asignados: ['D', 'I']
p: 0.01
rand: 0.23
clientes_por_camion: [[], ['B'], ['K']

In [441]:
print(costo_por_ton_repartida_min)
print(clientes_por_camion_min)
print(pedidos_no_asignados_min)

inf
[]
[]


In [442]:
for i in range(1000, 0, -1):
    print(i)


1000
999
998
997
996
995
994
993
992
991
990
989
988
987
986
985
984
983
982
981
980
979
978
977
976
975
974
973
972
971
970
969
968
967
966
965
964
963
962
961
960
959
958
957
956
955
954
953
952
951
950
949
948
947
946
945
944
943
942
941
940
939
938
937
936
935
934
933
932
931
930
929
928
927
926
925
924
923
922
921
920
919
918
917
916
915
914
913
912
911
910
909
908
907
906
905
904
903
902
901
900
899
898
897
896
895
894
893
892
891
890
889
888
887
886
885
884
883
882
881
880
879
878
877
876
875
874
873
872
871
870
869
868
867
866
865
864
863
862
861
860
859
858
857
856
855
854
853
852
851
850
849
848
847
846
845
844
843
842
841
840
839
838
837
836
835
834
833
832
831
830
829
828
827
826
825
824
823
822
821
820
819
818
817
816
815
814
813
812
811
810
809
808
807
806
805
804
803
802
801
800
799
798
797
796
795
794
793
792
791
790
789
788
787
786
785
784
783
782
781
780
779
778
777
776
775
774
773
772
771
770
769
768
767
766
765
764
763
762
761
760
759
758
757
756
755
754
753
752
751

In [ ]:
1)
            costo_por_ton_repartida_min = min(costo_por_ton_repartida, costo_por_ton_repartida_min)


In [ ]:


# def recocidoSimulado():
#     costo_por_ton = 10000
    
#     clientes_por_camion, pedidos_no_asignados = generarSolucionInicial()
#     pedidos_por_camion = pedidosPorCamion(clientes_por_camion)
#     tons_repartidas_por_camion, tons_no_repartidas = tonsAsignadasPorCamion(pedidos_por_camion = pedidos_por_camion)
#     tons_no_asignadas = tonsNoAsignadas(pedidos_no_asignados)
#     costo_por_camion = costoPorCamion(tons_repartidas_por_camion)

#     tons_totales = np.array(tons_repartidas_por_camion).sum() + tons_no_repartidas + tons_no_asignadas
#     tons_repartidas = np.array(tons_repartidas_por_camion)
#     tons_no_asignadas_o_no_repartidas = tons_no_repartidas + tons_no_asignadas
#     costo_por_camion = costoPorCamion(tons_repartidas_por_camion)


#     costo_total = costoTotal(costo_por_camion, tons_no_asignadas_o_no_repartidas)

#     tons_repartidas = tonsRepartidas(tons_repartidas_por_camion)

#     costo_por_ton_repartida = costo_total/tons_repartidas

#     return min_costo_por_tonelada, clientes_por_camion, pedidos_no_asignados


In [161]:
print(generarSolucionInicial(1)[0])
print(generarSolucionInicial(1)[1])

[['G'], ['J', 'F', 'D'], ['I'], ['H'], ['E', 'A'], ['L', 'K']]
['C', 'B']


In [ ]:
# pedidos_por_asignar

# clientes_por_camion = [[] for _ in range(CANTIDAD_DE_CAMIONES)] #Armo la lista con los pedidos de los camiones por cliente
# pedidos_no_asignados = []

# cantidad_de_clientes = len(pedidos_por_asignar)

# for i in range(cantidad_de_clientes): #Recorro toda la lista de clientes para ese dia
#     cliente = pedidos_por_asignar.pop() #Elimino el ultimo cliente de la lista de clientes a la vez que le asigno el cliente eliminado a la variable cliente
#     rand = random.randint(0, CANTIDAD_DE_CAMIONES-1) #genero un numero aleatorio entero del 0 al 5
#     camion_cliente = clientes_por_camion[rand]
#     cantidad_clientes = len(camion_cliente)
#     if cantidad_clientes < CANTIDAD_DE_CLIENTES_POR_CAMION: #Verifico que el camion no tenga 3 clientes
#         llega_list = []
#         for j in range(cantidad_clientes):
#             cliente_en_camion = camion_cliente[j]
#             llega = df_llega.loc[cliente_en_camion,cliente]
#             llega_list.append(llega)
#         llega_array = np.array(llega_list)
#         llega_todos = np.prod(llega_array)
#         #print(llega_list)
#         #print(prod)
#         if(llega_todos):
#             clientes_por_camion[rand].append(cliente)
#         else:
#             pedidos_no_asignados.append(cliente)

In [164]:
datos_prueba.loc["Total",dia]

47

In [120]:
clientes_camion = [[], [], ['B'], ['E', 'D', 'H'], [], ['J', 'K', 'L']]
pedidos_por_camion = []
for camion in clientes_camion: #Recorro los clientes por camion
    pedidos = datos_prueba.loc[camion,dia] #Selecciono los pedidos de ese cliente(las tons)
    pedidos_por_camion.append(pedidos)
print(pedidos_por_camion)
type(pedidos_por_camion[0])

[Series([], Name: 1, dtype: int64), Series([], Name: 1, dtype: int64), Cliente
B    6
Name: 1, dtype: int64, Cliente
E    4
D    8
H    5
Name: 1, dtype: int64, Series([], Name: 1, dtype: int64), Cliente
J    1
K    1
L    6
Name: 1, dtype: int64]


pandas.core.series.Series

In [119]:
pedidosPorCamion(clientes_camion)

pandas.core.series.Series

In [286]:
pedidos = datos_prueba[1]
pedidos = pedidos[pedidos > 0 ].index
pedidos = pedidos.to_list()
pedidos.pop()
pedidos

pedidos1 = datos_prueba[1]
pedidos1[['A', 'B']].values

array([4, 6], dtype=int64)

In [398]:
#DEBUG FUNCION GENERARVECINO1

clientes_por_camion = [[], [], ['J', 'I'], ['L', 'F'], ['K', 'H', 'G'], ['D', 'A']]
pedidos_por_asignar = ['E', 'C', 'B']

for i in range(CANTIDAD_DE_CAMIONES):
        clientes_por_camion_i = clientes_por_camion[i]#Seleccionamos los pedidos del camion i
        print("CAMION:", i + 1)
        len_i = len(clientes_por_camion_i) #Veo cuantos pedidos tiene el camion
        if(len_i > 0):
            rand = random.randint(0, len_i-1) #Selecciono el pedido a cambiar
            #clientes_por_camion_i[rand]
            cliente_eliminado = clientes_por_camion_i.pop(rand)
            print("eliminado", cliente_eliminado, "del camion", i+1)
            pedidos_por_asignar.append(cliente_eliminado)
            print("clientes_por_camion", clientes_por_camion)
            print("pedidos_por_asignar:", pedidos_por_asignar)

CAMION: 1
CAMION: 2
CAMION: 3
eliminado I del camion 3
clientes_por_camion [[], [], ['J'], ['L', 'F'], ['K', 'H', 'G'], ['D', 'A']]
pedidos_por_asignar: ['E', 'C', 'B', 'I']
CAMION: 4
eliminado F del camion 4
clientes_por_camion [[], [], ['J'], ['L'], ['K', 'H', 'G'], ['D', 'A']]
pedidos_por_asignar: ['E', 'C', 'B', 'I', 'F']
CAMION: 5
eliminado K del camion 5
clientes_por_camion [[], [], ['J'], ['L'], ['H', 'G'], ['D', 'A']]
pedidos_por_asignar: ['E', 'C', 'B', 'I', 'F', 'K']
CAMION: 6
eliminado A del camion 6
clientes_por_camion [[], [], ['J'], ['L'], ['H', 'G'], ['D']]
pedidos_por_asignar: ['E', 'C', 'B', 'I', 'F', 'K', 'A']


In [418]:
pedidos_por_asignar = ['E', 'C', 'B', 'F', 'K', 'A']
clientes_por_camion = [[], [], ['J'], ['L'], ['H', 'G','I'], ['D']]
cantidad_pedidos_por_asignar = len(pedidos_por_asignar)
pedidos_no_asignados = []
#FOR ITERA SOBRE TODOS LOS PEDIDOS POR ASIGNAR
for i in range(cantidad_pedidos_por_asignar):
    #ELIMINA EL ULTIMO CLIENTE DE LOS PEDIDOS Y LO ASIGNA A CLIENTE
    cliente_a_asignar = pedidos_por_asignar.pop()
    print("Cliente a asignar", cliente_a_asignar)
    #NUMERO ALEATORIO ENTRE 0,5
    rand = random.randint(0, CANTIDAD_DE_CAMIONES-1)
    #CAMION [] DE CLIENTE EN EL CAMION[RAND]
    camion = clientes_por_camion[rand]
    print("Camion", rand + 1, camion)
    cantidad_clientes = len(camion)
    if  cantidad_clientes < CANTIDAD_DE_CLIENTES_POR_CAMION:
        llega_list = []
        for j in range(cantidad_clientes):
            cliente_en_camion = camion[j]
            llega = df_llega.loc[cliente_en_camion,cliente_a_asignar]
            llega_list.append(llega)
        llega_array = np.array(llega_list)
        prod = np.prod(llega_array)

        if(prod == 1):
            clientes_por_camion[rand].append(cliente_a_asignar)
        else:
            pedidos_no_asignados.append(cliente_a_asignar)
    else:
        pedidos_no_asignados.append(cliente_a_asignar)

print("clientes_por_camion", clientes_por_camion)
print("pedidos_no_asignados:", pedidos_no_asignados )

Cliente a asignar A
Camion 1 []
Cliente a asignar K
Camion 6 ['D']
Cliente a asignar F
Camion 5 ['H', 'G', 'I']
Cliente a asignar B
Camion 6 ['D']
Cliente a asignar C
Camion 4 ['L']
Cliente a asignar E
Camion 5 ['H', 'G', 'I']
clientes_por_camion [['A'], [], ['J'], ['L', 'C'], ['H', 'G', 'I'], ['D', 'B']]
pedidos_no_asignados: ['K', 'F', 'E']


In [424]:
pedidos_por_asignar = ['E', 'C', 'B', 'F', 'K', 'A']
clientes_por_camion = [[], [], ['J'], ['L'], ['H', 'G','I'], ['D']]

clientes_por_camion, pedidos_no_asignados = generarVecino1(pedidos_no_asignados= pedidos_por_asignar, clientes_por_camion=clientes_por_camion)
print(clientes_por_camion)
print(pedidos_no_asignados)

[['F'], ['B'], ['K'], ['J'], ['G', 'I', 'E'], ['D', 'H']]
['L', 'A', 'C']


In [511]:
datos_prueba

,1,2,3,4,5,6,7,8,9,10
Cliente,,,,,,,,,,
A,4,3,5,2,6,6,6,5,5,1
B,6,6,5,4,1,3,7,6,4,7
C,1,6,6,2,6,7,5,1,4,5
D,8,6,5,2,8,2,2,2,5,7
E,4,1,2,4,2,3,2,6,3,6
F,3,1,6,8,6,5,5,3,5,8
G,3,2,3,7,4,3,3,8,4,4
H,5,8,5,2,4,8,4,2,7,1
I,5,2,3,4,2,6,1,4,5,7


Q&A

In [538]:
#GENERARSOLUCIONINICIAL 
import itertools
pedidos = datos_prueba[5]
lista_bool = []
for i in range(100):
    #RESULTADO RECIBIDO
    clientes_por_camion, pedidos_no_asignados = generarSolucionInicial(pedidos= pedidos)
    clientes_por_camion.append(pedidos_no_asignados)
    clientes_desordenados = list(itertools.chain.from_iterable(clientes_por_camion))
    clientes_ordenados = sorted(clientes_desordenados)
    #RESULTADO ESPERADO
    pedidos_index = pedidos[pedidos > 0 ].index
    lista_pedidos = pedidos_index.to_list()
    lista_pedidos.pop()
    boleano = lista_pedidos == clientes_ordenados
    lista_bool.append(boleano)


boleano_tot = np.array(lista_bool).prod()
print(boleano_tot)

1


In [ ]:
#PEDIDOSPORCAMION
